## 1. 환경 설정

`(1) Env 환경변수`

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [3]:
import os
from glob import glob

from pprint import pprint
import json

## 2. 벡터 저장소 (Vector Store)

### 2.1 Chroma

- 사용자 편의성이 우수한 오픈소스 벡터 저장소
 - `langchain-chroma` 패키지 설치

`(1) 벡터 저장소 초기화`

In [4]:
# 벡터 저장소에 문서를 저장할 때 적용할 임베딩 모델
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")



# 벡터 저장소 생성
from langchain_chroma import Chroma
chroma_db = Chroma(
    embedding_function=embeddings_model,
    collection_name="ai_sample_collection",
    persist_directory="./chroma_db",
)


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [5]:
chroma_db.get()

Failed to send telemetry event CollectionGetEvent: capture() takes 1 positional argument but 3 were given


{'ids': ['DOC_1', 'DOC_2', 'DOC_3', 'DOC_4'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다.',
  '머신러닝은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 하위 분야입니다.',
  '딥러닝은 머신러닝의 한 종류로, 심층 신경망을 사용하여 학습합니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 1', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 4', 'source': 'AI_textbook'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

`(2) 벡터 저장소 관리`  

- 문서 추가: `vector_store.add_documents(documents, ids)`

In [6]:
from langchain_core.documents import Document

# 문서 컬렉션
documents = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.",
    "컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다."
]

# Document 객체 생성
doc_objects = []
for i, content in enumerate(documents, start=1):
    doc = Document(
        page_content=content,
        metadata={"source":"AI_textbook", "chapter": f"Chapter {i}"},
    )
    doc_objects.append(doc)

# 순차적 ID 리스트 생성
doc_ids = [f"DOC_{i}" for i in range(1, len(doc_objects) + 1)]

# 문서를 벡터 저장소에 저장
added_doc_ids = chroma_db.add_documents(documents=doc_objects, ids=doc_ids)

# 벡터 저장소에 저장된 문서를 확인
print(f"{len(added_doc_ids)}개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.")
print(added_doc_ids)

5개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.
['DOC_1', 'DOC_2', 'DOC_3', 'DOC_4', 'DOC_5']


In [7]:
# 저장된 문서 검색
query = "인공지능과 머신러닝의 관계는?"
results = chroma_db.similarity_search(query, k=2)

print(f"\n쿼리: {query}")
print("가장 유사한 문서:")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given



쿼리: 인공지능과 머신러닝의 관계는?
가장 유사한 문서:
- 머신러닝은 인공지능의 하위 분야입니다. [출처: AI_textbook, Chapter 2]
- 딥러닝은 머신러닝의 한 종류입니다. [출처: AI_textbook, Chapter 3]


In [8]:
# 현재 저장된 컬렉션 데이터 확인
chroma_db.get()

{'ids': ['DOC_1', 'DOC_2', 'DOC_3', 'DOC_4', 'DOC_5'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 한 분야입니다.',
  '머신러닝은 인공지능의 하위 분야입니다.',
  '딥러닝은 머신러닝의 한 종류입니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.',
  '컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 1', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 4', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 5', 'source': 'AI_textbook'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

- 문서 수정: `vector_store.update_document(document_id, document)`

In [9]:
# 업데이트할 문서 생성
updated_document_1 = Document(
    page_content="인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다.",
    metadata={"source": "AI_textbook", "chapter": "Chapter 1"},
)

updated_document_2 = Document(
    page_content="머신러닝은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 하위 분야입니다.",
    metadata={"source": "AI_textbook", "chapter": "Chapter 2"},
)

updated_document_3 = Document(
    page_content="딥러닝은 머신러닝의 한 종류로, 심층 신경망을 사용하여 학습합니다.",
    metadata={"source": "AI_textbook", "chapter": "Chapter 3"},
)


# 단일 문서 업데이트
chroma_db.update_document(document_id="DOC_1", document=updated_document_1)

# 여러 문서 한 번에 업데이트
chroma_db.update_documents(ids=["DOC_2", "DOC_3"], documents=[updated_document_2, updated_document_3])

print("문서 업데이트 완료")

Failed to send telemetry event CollectionUpdateEvent: capture() takes 1 positional argument but 3 were given


문서 업데이트 완료


In [10]:
# 저장된 문서 검색 예시
query = "인공지능과 머신러닝의 관계는?"
results = chroma_db.similarity_search(query, k=2)

print(f"\n쿼리: {query}")
print("가장 유사한 문서:")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")


쿼리: 인공지능과 머신러닝의 관계는?
가장 유사한 문서:
- 머신러닝은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 하위 분야입니다. [출처: AI_textbook, Chapter 2]
- 인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다. [출처: AI_textbook, Chapter 1]


- 문서 삭제: `vector_store.delete(ids)`

In [11]:
chroma_db.delete(ids=["DOC_5"])

Failed to send telemetry event CollectionDeleteEvent: capture() takes 1 positional argument but 3 were given


In [12]:
# 컬렉션 확인
chroma_db.get()

{'ids': ['DOC_1', 'DOC_2', 'DOC_3', 'DOC_4'],
 'embeddings': None,
 'documents': ['인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다.',
  '머신러닝은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 하위 분야입니다.',
  '딥러닝은 머신러닝의 한 종류로, 심층 신경망을 사용하여 학습합니다.',
  '자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'chapter': 'Chapter 1', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 2', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 3', 'source': 'AI_textbook'},
  {'chapter': 'Chapter 4', 'source': 'AI_textbook'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

`(3) 문서 검색`  

- 유사도 검색
    - 주어진 쿼리와 가장 유사한 문서를 반환
    -  k=2는 상위 2개의 결과를 반환하도록 지정
    - filter를 사용하여 특정 출처의 문서만 검색 가능

In [13]:
query = "인공지능과 머신러닝의 차이점은 무엇인가요?"
results = chroma_db.similarity_search(
    query,
    k=2,
    filter={"source": "AI_textbook"}
)

print("유사도 검색 결과:")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")

유사도 검색 결과:
- 머신러닝은 데이터로부터 학습하여 예측과 결정을 내리는 인공지능의 하위 분야입니다. [출처: AI_textbook, Chapter 2]
- 인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다. [출처: AI_textbook, Chapter 1]


- 유사도 점수가 포함된 검색
    - 유사도 점수를 함께 반환
    - 점수가 낮을수록 더 유사한 것을 의미 (거리 기준으로 점수가 산정되기 때문)

In [14]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = chroma_db.similarity_search_with_score(
    query,
    k=2,
    filter={"source": "AI_textbook"}
)

print("점수가 포함된 유사도 검색 결과:\n")
for doc, score in results:
    print(f"- 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")
    print()

점수가 포함된 유사도 검색 결과:

- 점수: 0.5772
  내용: 딥러닝은 머신러닝의 한 종류로, 심층 신경망을 사용하여 학습합니다.
  [출처: AI_textbook, Chapter 3]

- 점수: 0.7292
  내용: 인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다.
  [출처: AI_textbook, Chapter 1]



- 관련성 점수가 포함된 검색
    - 문서와 함께 0에서 1 사이의 관련성 점수를 반환
    - 0은 가장 관련성이 낮고, 1은 가장 관련성이 높음을 의미

In [15]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = chroma_db.similarity_search_with_relevance_scores(
    query,
    k=2,
    filter={"source": "AI_textbook"}
)

print(f"쿼리: {query}")
print("\n검색 결과 (관련성 점수 포함):")
for doc, score in results:
    print(f"- 관련성 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")
    print()

쿼리: 딥러닝은 어떤 분야에서 사용되나요?

검색 결과 (관련성 점수 포함):
- 관련성 점수: 0.5919
  내용: 딥러닝은 머신러닝의 한 종류로, 심층 신경망을 사용하여 학습합니다.
  [출처: AI_textbook, Chapter 3]

- 관련성 점수: 0.4844
  내용: 인공지능은 컴퓨터 과학의 핵심 분야 중 하나로, 기계학습과 딥러닝을 포함합니다.
  [출처: AI_textbook, Chapter 1]



`(4) 벡터 저장소 로드`  

In [16]:
chroma_db2 = Chroma(
    collection_name="ai_smaple_collection",
    embedding_function=embeddings_model,
    persist_directory="./chroma_db",
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [17]:
# 미리 임베딩된 쿼리 벡터를 사용하여 검색
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = chroma_db2.similarity_search_with_relevance_scores(
    query,
    k=2,
    filter={"source": "AI_textbook"}
)

print(f"쿼리: {query}")
print("\n검색 결과 (관련성 점수 포함):")
for doc, score in results:
    print(f"- 관련성 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")
    print()

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


쿼리: 딥러닝은 어떤 분야에서 사용되나요?

검색 결과 (관련성 점수 포함):


### 2.2 FAISS(Facebook AI Similarity Search)

- 효율적인 벡터 유사도 검색 및 클러스터링을 위한 오픈소스 벡터 저장소 
- `faiss-cpu` 패키지 설치

`(1) 벡터 저장소 초기화`

In [18]:
# 벡터 저장소에 문서를 저장할 때 적용할 임베딩 모델
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
embeddings_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# 벡터 저장소 생성
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

# FAISS 인덱스 초기화 (유클리드 거리 사용)
faiss_index = faiss.IndexFlatL2(len(embeddings_model.embed_query("hello world")))
print("FAISS 인덱스 초기화 완료")

FAISS 인덱스 초기화 완료


In [19]:
# FAISS 벡터 저장소의 벡터 차원 수 (임베딩 차원 수)
faiss_index.d

1024

In [20]:
# FAISS 벡터 저장소 생성
faiss_db = FAISS(
    embedding_function=embeddings_model,
    index=faiss_index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)
# 저장된 문서의 갯수 확인
faiss_db.index.ntotal

0

`(2) 벡터 저장소 관리`  

- 문서 추가: `vector_store.add_documents(documents, ids)`

In [21]:
from langchain_core.documents import Document

# 문서 컬렉션
documents = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.",
    "컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다."
]

# Document 객체 생성
doc_objects = []
for i, content in enumerate(documents, start=1):
    doc = Document(
        page_content=content,
        metadata={"source": "AI_textbook", "chapter": f"Chapter {i}"},
    )
    doc_objects.append(doc)


# 순차적 ID 리스트 생성
doc_ids = [f"DOC_{i}" for i in range(1, len(doc_objects) + 1)]

# 문서를 벡터 저장소에 저장
added_doc_ids = faiss_db.add_documents(documents=doc_objects, ids=doc_ids)

# 벡터 저장소에 저장된 문서를 확인
print(f"{len(added_doc_ids)}개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.")
print(added_doc_ids)

5개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.
['DOC_1', 'DOC_2', 'DOC_3', 'DOC_4', 'DOC_5']


In [22]:
# 저장된 문서의 갯수 확인
faiss_db.index.ntotal

5

- 문서 삭제: `vector_store.delete(ids)`

In [23]:
faiss_db.delete(ids=["DOC_5"])

True

In [24]:
# 컬렉션 확인
faiss_db.index.ntotal

4

`(3) 문서 검색`  

- 유사도 검색
    - 주어진 쿼리와 가장 유사한 문서를 반환
    - k=2는 상위 2개의 결과를 반환하도록 지정
    - filter를 사용하여 특정 출처의 문서만 검색 가능

In [25]:
query = "인공지능과 머신러닝의 차이점은 무엇인가요?"
results = faiss_db.similarity_search(
    query,
    k=2,
    filter={"source": "AI_textbook"}
)

print("유사도 검색 결과:")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")

유사도 검색 결과:
- 머신러닝은 인공지능의 하위 분야입니다. [출처: AI_textbook, Chapter 2]
- 딥러닝은 머신러닝의 한 종류입니다. [출처: AI_textbook, Chapter 3]


- 유사도 점수가 포함된 검색
    - 유사도 점수를 함께 반환
    - 점수가 낮을수록 더 유사한 것을 의미

In [26]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = faiss_db.similarity_search_with_score(
    query,
    k=2,
    filter={"source": "AI_textbook"}
)

print("점수가 포함된 유사도 검색 결과:\n")
for doc, score in results:
    print(f"- 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")
    print()

점수가 포함된 유사도 검색 결과:

- 점수: 0.6517
  내용: 딥러닝은 머신러닝의 한 종류입니다.
  [출처: AI_textbook, Chapter 3]

- 점수: 0.8442
  내용: 머신러닝은 인공지능의 하위 분야입니다.
  [출처: AI_textbook, Chapter 2]



- 관련성 점수가 포함된 검색
    - 문서와 함께 0에서 1 사이의 관련성 점수를 반환
    - 0은 가장 관련성이 낮고, 1은 가장 관련성이 높음을 의미

In [27]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = faiss_db.similarity_search_with_relevance_scores(
    query,
    k=2,
    filter={"source": "AI_textbook"}
)

print(f"쿼리: {query}")
print("\n검색 결과 (관련성 점수 포함):")
for doc, score in results:
    print(f"- 관련성 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")
    print()

쿼리: 딥러닝은 어떤 분야에서 사용되나요?

검색 결과 (관련성 점수 포함):
- 관련성 점수: 0.5392
  내용: 딥러닝은 머신러닝의 한 종류입니다.
  [출처: AI_textbook, Chapter 3]

- 관련성 점수: 0.4031
  내용: 머신러닝은 인공지능의 하위 분야입니다.
  [출처: AI_textbook, Chapter 2]



`(4) 로컬에 저장 및 로드`  

In [28]:
# 로컬에 저장
faiss_db.save_local("faiss_ai_smaple_index")

In [29]:
# 로컬에 저장된 FAISS 벡터 저장소 불러오기
faiss_db2 = FAISS.load_local(
    "faiss_ai_smaple_index", embeddings_model, allow_dangerous_deserialization=True
)

In [30]:
# 저장된 문서 객체를 확인
faiss_db2.docstore._dict

{'DOC_1': Document(metadata={'source': 'AI_textbook', 'chapter': 'Chapter 1'}, page_content='인공지능은 컴퓨터 과학의 한 분야입니다.'),
 'DOC_2': Document(metadata={'source': 'AI_textbook', 'chapter': 'Chapter 2'}, page_content='머신러닝은 인공지능의 하위 분야입니다.'),
 'DOC_3': Document(metadata={'source': 'AI_textbook', 'chapter': 'Chapter 3'}, page_content='딥러닝은 머신러닝의 한 종류입니다.'),
 'DOC_4': Document(metadata={'source': 'AI_textbook', 'chapter': 'Chapter 4'}, page_content='자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.')}

In [31]:
query = "딥러닝은 어떤 분야에서 사용되나요?"
results = faiss_db2.similarity_search_with_relevance_scores(
    query,
    k=2,
    filter={"source": "AI_textbook"}
)

print(f"쿼리: {query}")
print("\n검색 결과 (관련성 점수 포함):")
for doc, score in results:
    print(f"- 관련성 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f"  [출처: {doc.metadata['source']}, {doc.metadata['chapter']}]")
    print()

쿼리: 딥러닝은 어떤 분야에서 사용되나요?

검색 결과 (관련성 점수 포함):
- 관련성 점수: 0.5392
  내용: 딥러닝은 머신러닝의 한 종류입니다.
  [출처: AI_textbook, Chapter 3]

- 관련성 점수: 0.4031
  내용: 머신러닝은 인공지능의 하위 분야입니다.
  [출처: AI_textbook, Chapter 2]



## 3. RAG 검색기
### 3.1 Semantic Search(의미론적 검색)
- Vector Store 검색기 사용   

(1) 벡터 저장소 초기화   
- cosine distance 기준으로 인덱싱

In [34]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from transformers import AutoTokenizer

#데이터 로드
def load_text_files(txt_files):
    data = []

    for text_file in txt_files:
        loader = TextLoader(text_file, encoding='utf-8')
        data += loader.load()

    return data

korean_txt_files = glob(os.path.join('data', '*_KR.txt'))
korean_data = load_text_files(korean_txt_files)

# Hugging Face의 임베딩 모델 토크나이저 지정
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

# 문장을 구분하여 분할
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    separator=r"[.!?]\s+",
    chunk_size=100,
    chunk_overlap=0,
    is_separator_regex=True,
    keep_separator=True,
)

korean_docs = text_splitter.split_documents(korean_data)

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# Hugging Face의 임베딩 모델 생성
embeddings_huggingface = HuggingFaceEmbeddings(model_name = "BAAI/bge-m3")

# Chroma 벡터 저장소 불러오기
chroma_db = Chroma.from_documents(
    embedding=embeddings_huggingface,
    documents=korean_docs,
    collection_name="db_korean_cosine",
    persist_directory="./chroma.db",
    collection_metadata={'hnsw:space': 'cosine'}, # l2, ip, cosine 중에서 선택
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


(2) Top K

In [39]:
chroma_k_retriever = chroma_db.as_retriever(
    search_kwargs={"k":2},
)

query = "리비안은 언제 사업을 시작했나요?"
retrieved_docs = chroma_k_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


쿼리: 리비안은 언제 사업을 시작했나요?
검색 결과:
- 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다 [출처: data\리비안_KR.txt]
- .

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다 [출처: data\리비안_KR.txt]


(3) 임계값 지정   
- Similarity score threshold (기준 스코어 이상인 문서를 대상으로 추출)

In [44]:
from langchain_community.utils.math import cosine_similarity

chroma_threshold_retriever = chroma_db.as_retriever(
    search_type='similarity_score_threshold',
    search_kwargs={'score_threshold': 0.6, 'k':2}
)

query = "리비안은 언제 사업을 시작했나요?"
retrieved_docs = chroma_threshold_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    score = cosine_similarity(
        [embeddings_huggingface.embed_query(query)],
        [embeddings_huggingface.embed_query(doc.page_content)]
    )[0][0]
    print(f"- {doc.page_content} [유사도: {score:.4f}]")


쿼리: 리비안은 언제 사업을 시작했나요?
검색 결과:
- 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다 [유사도: 0.6734]


(4) MMR(Maximal Marginal Relevance) 검색

In [51]:
# MMR - 다양성 고려 (LAMDA_MULT 작을수록 더 다양하게 추출)
chroma_mmr = chroma_db.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k':3, # 검색할 문서 수
        'fetch_k':8, # mmr 알고리즘에 전달할 문서의 수 (fetch_k > k)
        'lambda_mult':0.5 # 다양성을 고려하는 정도 (1은 최소 다양성, 0은 최대 다양성을 의미, 기본값은 0.5)
    }
)

query = "리비안은 언제 사업을 시작했나요?"
retrieved_docs = chroma_mmr.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")

for doc in retrieved_docs:
    score = cosine_similarity(
        [embeddings_huggingface.embed_query(query)],
        [embeddings_huggingface.embed_query(doc.page_content)]
    )[0][0]
    print(f"- {doc.page_content} [유사도: {score:.4f}]")
    

쿼리: 리비안은 언제 사업을 시작했나요?
검색 결과:
- 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다 [유사도: 0.6734]
- .

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다 [유사도: 0.5955]
- .

리비안은 2021년 10월 첫 번째 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다. [유사도: 0.5690]


(5) metadata 필터링 검색

In [60]:
# 문서 객체의 metadata를 이용한 필터링
chrom_metadata = chroma_db.as_retriever(
    search_kwargs={
        'filter': {'source': 'data\리비안_KR.txt'},
        'k': 2, 
        }
)

query = "리비안은 언제 사업을 시작했나요?"
retrieved_docs = chrom_metadata.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")

쿼리: 리비안은 언제 사업을 시작했나요?
검색 결과:
- 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다 [출처: data\리비안_KR.txt]
- .

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다 [출처: data\리비안_KR.txt]


(6) page_content 본문 필터링 검색

In [55]:
# page_content를 이용한 필터링
chroma_content = chroma_db.as_retriever(
    search_kwargs={
        'k':2,
        'where_document':{'$contains':'리비안'}
    }
)

query = "리비안은 언제 사업을 시작했나요?"
retrieved_docs = chroma_content.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")

쿼리: 리비안은 언제 사업을 시작했나요?
검색 결과:
- 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다 [출처: data\리비안_KR.txt]
- .

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다 [출처: data\리비안_KR.txt]
